In [ ]:
#create a nerf model
input_ch = 3
#? What's input_ch_views?
input_ch_views = 3
import tensorflow as tf

inputs = tf.keras.Input(shape = input_ch + input_ch_views)

inputs_pts, inputs_views = tf.split(inputs, [input_ch, input_ch_views], -1)
# display(inputs_pts.shape, ",", inputs_views.shape)
inputs_pts.set_shape([None, input_ch])
inputs_views.set_shape([None, input_ch_views])
# display(inputs_pts.shape, ",", inputs_views.shape)

#outputs: why does output equal to inputs_pts
outputs = inputs_pts
D = 8
W = 256
relu = tf.keras.layers.ReLU()
def dense(W, act=relu): return tf.keras.layers.Dense(W, activation=act)
for i in range(8):
    outputs = dense(W)(outputs)
    if i in [4]:
        outputs = tf.concat([inputs_pts, outputs], -1)
useViewDirs = True
if useViewDirs:
    # what are these outputs?
    alpha_out = dense(1, act=None)(outputs)
    bottleneck = dense(256, act=None)(outputs)

    input_viewdirs = tf.concat([bottleneck, inputs_views], -1)
    outputs = input_viewdirs
    for i in range(1):
        outputs = dense(W//2)(outputs)
    outputs = dense(3, act = None)(outputs)
    outputs = tf.concat([outputs, alpha_out], -1)
else:
    output_ch = 4
    outputs = dense(output_ch, act = None)(outputs)
model = tf.keras.Model(inputs, outputs)
print(model.summary())

grad_vars = model.trainable_variables
display(grad_vars)
# models = {"model": model}




In [15]:
import tensorflow as tf
z_vals = tf.linspace(0., 1., 4)
z_vals_new = tf.broadcast_to(z_vals, [5,4])
display(z_vals)
display(z_vals_new)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.        , 0.33333334, 0.6666667 , 1.        ], dtype=float32)>

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[0.        , 0.33333334, 0.6666667 , 1.        ],
       [0.        , 0.33333334, 0.6666667 , 1.        ],
       [0.        , 0.33333334, 0.6666667 , 1.        ],
       [0.        , 0.33333334, 0.6666667 , 1.        ],
       [0.        , 0.33333334, 0.6666667 , 1.        ]], dtype=float32)>

In [3]:
import tensorflow as tf
import numpy as np

# a = np.random.randint(0,10, size=(3,4))
# print(a)
# # norms = tf.linalg.norm(a, axis=-1)
# # print(norms)
# cump = tf.math.cumprod(a, axis=-1)
# print(cump)

display([2,3] + [9,10])

[2, 3, 9, 10]

In [5]:
from nerf.run_nerf_helpers import sample_pdf, img2mse, mse2psnr
from nerf.run_nerf import create_nerf, config_parser
import numpy as np
import tensorflow as tf

from load_llff import load_llff_data
data_dir = "./data/nerf_llff_data/fern/"
# get rays

def render_rays(ray_batch, network_query_fn, network_fn, N_samples, retraw=False, N_importance=10, lindisp=False, perturb=0., network_fine=None, white_bkgd=False, raw_noise_std = 0., verbose=False):

    def raw2outputs(raw, z_vals, rays_d):
        def raw2alpla(raw, dists, act_fn = tf.nn.relu):
            return 1.0 - tf.exp(-act_fn(raw) * dists)
        # ???
        dists = z_vals[..., 1:] - z_vals[..., -1:]
        # ???
        dists = tf.concat([dists, tf.broadcast_to([1e10], dists[..., :1].shape)], axis=-1)
        dists = dists * tf.linalg.norm(rays_d[..., None, :], axis=-1)
        rgb = tf.math.sigmoid(raw[..., :3])

        noise = 0.
        if(raw_noise_std > 0):
            noise = tf.random.normal(raw[..., :3].shape) * raw_noise_std
        #     ???
        alpha = raw2alpla(raw[..., 3] + noise, dists)
        # ???
        weights = alpha * tf.math.cumprod(1.-alpha + 1e-10, axis=-1, exclusive=True)
        rgb_map = tf.reduce_sum(weights[..., None] * rgb, axis=-2)
        depth_map = tf.reduce_sum(weights * z_vals, axis=-1)
        disp_map = 1./tf.maximum(1e-10, depth_map/tf.reduce_sum(weights, axis=-1))
        acc_map = tf.reduce_sum(weights, axis=-1)
        if white_bkgd:
            rgb_map = rgb_map + (1 - acc_map[..., None])
        return rgb_map, disp_map, acc_map, weights, depth_map

    N_rays = ray_batch.shape[0]
    rays_o, rays_d = ray_batch[:, 0:3], ray_batch[:, 3:6]
    # ???
    viewdirs = ray_batch[:, -3:] if ray_batch.shape[-1] > 8 else None
    bounds = tf.reshape(ray_batch[..., 6:8], [-1, 1, 2])
    near, far = bounds[..., 0], bounds[..., 1]

    t_vals = tf.linspace(0., 1., N_samples)
    if not lindisp:
        z_vals = near * (1.-t_vals) + far * (t_vals)
    else:
        z_vals = 1 / (1/near * (1.-t_vals) + 1/far * (t_vals))
    z_vals = tf.broadcast_to(z_vals, [N_rays, N_samples])
    # print(z_vals.shape)
    # if perturb > 0.:
        # todo
    rays_d_new = rays_d[..., None, :]
    z_vals_new = z_vals[..., : , None]
    # print(rays_d_new * z_vals_new)

    pts = rays_o[..., None, :] + rays_d[..., None, :] * z_vals[..., : , None]
    raw = network_query_fn(pts, viewdirs, network_fn)
    rgb_map, disp_map, acc_map, weights, depth_map = raw2outputs(
        raw, z_vals, rays_d)
    if N_importance > 0:
        rgb_map_0, disp_map_0, acc_map_0 = rgb_map, disp_map, acc_map
        z_vals_mid = .5 * (z_vals[..., 1:] + z_vals[..., -1:])
        z_samples = sample_pdf(z_vals_mid, weights[..., 1:-1], N_importance, det = (perturb == 0.))
        z_samples = tf.stop_gradient(z_samples)
        z_vals = tf.sort(tf.concat([z_vals, z_samples], -1), -1)
        pts = rays_o[..., None, :] + rays_d[..., None, :] + z_vals[..., :, None]

        run_fn = network_fn if network_fine is None else network_fine
        raw = network_query_fn(pts, viewdirs, run_fn)
        rgb_map, disp_map, acc_map, weights, depth_map = raw2outputs(
            raw, z_vals, rays_d)

    ret = {'rgb_map': rgb_map, 'disp_map': disp_map, 'acc_map': acc_map}

    if retraw:
        ret['raw'] = raw
    if N_importance > 0:
        ret['rgb0'] = rgb_map_0
        ret['disp0'] = disp_map_0
        ret['acc0'] = acc_map_0
        ret['z_std'] = tf.math.reduce_std(z_samples, -1)  # [N_rays]

    for k in ret:
        tf.debugging.check_numerics(ret[k], 'output {}'.format(k))

    return ret

def batchify_rays(rays_flat, chunk, **kwargs):
    all_ret = {}
    for i in range(0, rays_flat.shape[0], chunk):
        ret = render_rays(rays_flat[i:i+chunk], **kwargs)
        for k in ret:
            if k not in all_ret:
                all_ret[k] = []
            all_ret[k].append(ret[k])
    all_ret = {k: tf.concat(all_ret[k], 0) for k in all_ret}
    return all_ret

def get_rays(H, W, focal, c2w):
    i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
    dirs = tf.stack([(i - W*.5)/focal, -(j - H*.5)/focal, -tf.ones_like(i)], -1)
    print(c2w)
    # Why needs to transform from camera to world
    rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3, :3], -1)
    # why origin is the c2w?
    rays_o = tf.broadcast_to(c2w[:3, -1], np.shape(rays_d))
    # print("c2w[:3, -1] shape: ", c2w[:3, -1], ", rays_o shape: ", rays_o.shape)
    return rays_d, rays_o

def render(H, W, focal, chunk=1024, rays=None, c2w=None, ndc=None, near=0., far=1., use_viewdirs=False, c2w_staticcam=None, **kwargs):
    if c2w:
        rays_d, rays_o = get_rays(H, W, focal, c2w)
    else:
        rays_d, rays_o = rays
    if use_viewdirs:
        viewdirs = rays_d
        if c2w_staticcam:
            rays_d, rays_o = get_rays(H, W, focal, c2w_staticcam)
        viewdirs = viewdirs / tf.linalg.norm(viewdirs, axis=-1, keepdims=True)
        viewdirs = tf.cast(tf.reshape(viewdirs, [-1,3]), dtype=tf.float32)
    sh = rays_d.shape
#        todo ndc
    rays_o = tf.cast(tf.reshape(rays_o, [-1, 3]), dtype=tf.float32)
    rays_d = tf.cast(tf.reshape(rays_d, [-1, 3]), dtype=tf.float32)

    near = near * tf.ones_like(rays_d[..., :1])
    far = far * tf.ones_like(rays_d[..., :1])
    rays = tf.concat([rays_o, rays_d, near, far], axis=-1)
    # display("rays.shape: ", rays.shape, "viewdirs.shape", viewdirs.shape)
    if use_viewdirs:
        rays = tf.concat([rays, viewdirs], axis=-1)
    all_ret = batchify_rays(rays, chunk, **kwargs)
    for k in all_ret:
        # ???
        k_sh = list(sh[:-1]) + list(all_ret[k].shape[1:])
        all_ret[k] = tf.reshape(all_ret[k], k_sh)
    k_extract = ['rgb_map', 'disp_map', 'acc_map']
    ret_list = [all_ret[k] for k in k_extract]
    ret_dict = {k:all_ret[k] for k in all_ret if k not in k_extract}
    return ret_list + [ret_dict]

def normalize(x):
    return x / np.linalg.norm(x)

def viewmatrix(z, up, pos):
    vec2 = normalize(z)
    vec1_avg = up
    vec0 = normalize(np.cross(vec1_avg, vec2))
    vec1 = normalize(np.cross(vec2, vec0))
    m = np.stack([vec0, vec1, vec2, pos], 1)
    return m


def get_rays_np(H, W, focal, c2w):
    """ray orgin, ray direction"""
    i, j = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')
    # here calculate direction
    dirs = np.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -np.ones_like(i)], -1)
    # todo 2 arrays have different shape, numpy do broadcasting
    rays_d = np.sum(dirs[..., np.newaxis, :] * c2w[:3, :3], -1)
    rays_o = np.broadcast_to(c2w[:3, -1], np.shape(rays_d))
    return rays_o, rays_d

def train():
    # create nerf model
    parser = config_parser()
    args = parser.parse_args()
    render_kwargs_train, render_kwargs_test, start, grad_vars, models = create_nerf(
            args)
    lrate = args.lrate
    optimizer = tf.keras.optimizers.Adam(lrate)
    models['optimizer'] = optimizer

    images, poses, bds, render_poses, i_test = load_llff_data(data_dir, 8,
                                                                      recenter=True, bd_factor=.75,
                                                                      spherify=True)

    chunk = 1024
    hwf = poses[0, :3, -1]
    H, W, focal = hwf
    H, W = int(H), int(W)
    hwf = [H, W, focal]

    # display("c2w: ", poses[:, :3, :4])
    #todo what's c2w=poses[:, :3, :4]?
    rays = [get_rays_np(H, W, focal, p) for p in poses[:, :3, :4]] #return 2 kinds of data of 20 picturs': 1. directions 2. direction origins
    rays = np.stack(rays, axis=0)
    # rays.shape: 20, 2, 378, 504, 3

    rays_rgb = np.concatenate([rays, images[:, None, ...]], 1)
    # display("rays.shape: ", rays.shape, ", rays_rgb.shape: ", rays_rgb.shape): (20, 2, 378, 504, 3), (20, 3, 378, 504, 3)
    rays_rgb = np.transpose(rays_rgb, [0, 2, 3, 1, 4])
    # print(rays_rgb.shape): 20 pictures, every pixel, channel(3), origin(3), direction(3)

    # slicing, so 0, 8, 16  are for testing
    i_test = np.arange(images.shape[0])[::8]
    i_val = i_test
    i_train = np.array([i for i in np.arange(int(images.shape[0])) if
                                (i not in i_test and i not in i_val)])

    rays_rgb = np.stack([rays_rgb[i] for i in i_train], axis=0)
    # display("rays_rgb shape", rays_rgb.shape): (17, 378, 504, 3, 3)
    rays_rgb = np.reshape(rays_rgb, [-1, 3, 3])
    # display("rays_rgb shape", rays_rgb.shape) : (3238704, 3, 3)

    rays_rgb = rays_rgb.astype(np.float32)
    np.random.shuffle(rays_rgb)

    i_batch = 0
    N_iters = 1
    N_rand = 5
    use_batching = not args.no_batching

    for i in range(start, N_iters):
        # here always use batch
        batch = rays_rgb[i_batch:i_batch + N_rand]
        batch = np.transpose(batch, [1, 0, 2])
        # what's target?
        batch_rays, target_s = batch[:2], batch[2]

        i_batch += N_rand
        if i_batch > rays_rgb.shape[0]:
            np.random.shuffle(rays_rgb)
            i_batch = 0
        with tf.GradientTape() as tape:
            render_kwargs_train['N_importance'] = 10
            rgb, disp, acc, extras =render(H, W, focal, chunk, batch_rays, **render_kwargs_train)
            img_loss = img2mse(rgb, target_s)
            # ????s
            trans = extras["raw"][..., -1]
            loss = img_loss
            psnr = mse2psnr(img_loss)

            if 'rgb0' in extras:
                img_loss0 = img2mse(extras['rgb0'], target_s)
                loss += img_loss0
                psnr0 = mse2psnr(img_loss0)

            gradients = tape.gradient(loss, grad_vars)
            optimizer.apply_gradients(zip(gradients, grad_vars))

        # display("batch: ", batch, ", batch_rays: ", batch_rays, ", target_s", target_s)
        break
train()

MODEL 63 0 <class 'int'> <class 'int'> False
(None, 63) (None, 63) (None, 0)


KeyboardInterrupt: 